In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import sys
import glob
import pandas as pd
import os
import seaborn as sns

from tqdm import tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import pickle
import re
import json
from pathlib import Path


from open_spiel.python.algorithms.exploitability import nash_conv, best_response
from open_spiel.python.examples.ubc_plotting_utils import *
from open_spiel.python.examples.ubc_sample_game_tree import sample_game_tree, flatten_trees, flatten_tree
from open_spiel.python.examples.ubc_clusters import projectPCA, projectUMAP, fitGMM

from auctions.webutils import *



import bokeh
from bokeh.layouts import row, column
from bokeh.plotting import figure, show, output_file, save
from bokeh.io import output_notebook
from bokeh.models import HoverTool, ColumnDataSource, ColorBar, LogColorMapper, LinearColorMapper
from bokeh.transform import linear_cmap, log_cmap
from bokeh.palettes import Magma256
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

output_notebook()


2022-05-19 18:42:35.414917: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-19 18:42:35.414944: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Loading BokehJS ...

In [3]:
EXPERIMENT_NAME = 'sats_earnest'
RUN_NAME = 'sats_3regions_3licenses_5types_hide-may17mlphalfsmallbuf-100'


checkpoint = get_checkpoint_by_name(EXPERIMENT_NAME, RUN_NAME)
game = load_game(checkpoint.game)
env_and_model = db_checkpoint_loader(checkpoint)

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/sats_3regions_3licenses_5types_hide.json
Done config parsing
Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/sats_3regions_3licenses_5types_hide.json
Done config parsing


In [8]:
s = get_first_actionable_state(game)
s.apply_action(2)
s.apply_action(2)
s.apply_action(2)
s.apply_action(2)
s.information_state_string()
data_lines = '\nProcessed:\n0, 0, 2\nAggregates:\n-, -, +\n'

In [9]:
data_lines.split('\n')

['', 'Processed:', '0, 0, 2', 'Aggregates:', '-, -, +', '']

In [ ]:
# EXPERIMENT_NAME = 'infocompare4'
# RUN_NAME = 'large_game_2-mar17lstm-100'
# EXPERIMENT_NAME = 'mar15_brier_lr'
# RUN_NAME = 'large_game_2-mar14mlp_3-100'
# EXPERIMENT_NAME = 'mar14_very_large_game_2'
# RUN_NAME = 'very_large_game_2-feb22mlp-100'

# EXPERIMENT_NAME = 'mar22'
# RUN_NAME = 'large_game_2-mar17lstm-102'


# checkpoint = get_checkpoint_by_name(EXPERIMENT_NAME, RUN_NAME)
# game = load_game(checkpoint.game)
# env_and_model = db_checkpoint_loader(checkpoint)

In [ ]:
checkpoint

In [ ]:
trees = sample_game_tree(env_and_model, num_samples=1000, seed=1, include_embeddings=True)

In [ ]:
# TODO: Move this to function?
# Embeddings
df = flatten_trees(trees).query('embedding.notna()', engine='python')
for player in range(1):
    dfp = df.query(f'player_id == {player}')
    embeddings = np.stack(dfp['embedding'].values).squeeze()

    # Reduce embedding to 2D with PCA
    pca, variance = projectPCA(embeddings)
    dfp['pca_0'] = pca[:, 0]
    dfp['pca_1'] = pca[:, 1]

    # Try all numeric columns
    numerics = ['category', 'int8', 'int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    newdf = dfp.select_dtypes(include=numerics)
    IGNORE = ['type', 'depth', 'player_id', 'num_plays', 'pct_plays', 'pca_0', 'pca_1', 'umap_0', 'umap_1']
    plots = []
    for k in newdf.columns:
        if k not in IGNORE:
            plot = plot_embedding(dfp, color_col=k, reduction_method='pca')
            plots.append(plot)



In [ ]:
# plots_to_html(plots, 'rounds.html')
# plots_to_html(plots, 'rounds_only2.html')

In [ ]:
for plot in plots:
    show(plot)
    # break


In [ ]:
df = flatten_trees(trees)
df.head()

In [ ]:
df0 = df.query('player_id == 1').copy()
embeddings = np.stack(df0['embedding'].values).squeeze()
display(embeddings.shape)

In [ ]:
pca, variance = projectPCA(embeddings)
df0['pca_0'] = pca[:, 0]
df0['pca_1'] = pca[:, 1]

In [ ]:
umap = projectUMAP(embeddings, n_neighbors=200, min_dist=0.8)
df0['umap_0'] = umap[:, 0]
df0['umap_1'] = umap[:, 1]

In [ ]:
gmm, clusters, scores = fitGMM(embeddings, verbose=True)

In [ ]:
plt_x, plt_y = list(zip(*list(scores.items())))
plt.plot(plt_x, plt_y)
plt.xlabel('Number of clusters')
plt.ylabel('AIC score')

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(
    x=df0['pca_0'],
    y=df0['pca_1'],
    c=clusters,
    cmap='tab20',
    s=50,
    alpha=0.5
)
# plt.colorbar()
plt.show()

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(
    x=df0['umap_0'],
    y=df0['umap_1'],
    c=clusters,
    cmap='tab20',
    s=50,
    alpha=0.5
)
# plt.colorbar()

In [ ]:
from sklearn import tree

# TODO: Need access to all of the features in the information state tensor if we really want to know what it's paying attention to. So that means storing feature vectors

X = df0['feature_vector'].values
X = np.array([np.array(x) for x in X])
Y = clusters
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X, Y)



In [ ]:
bundles = action_to_bundles(env_and_model.game_config['licenses'])
num_products = len(env_and_model.game_config['licenses'])
FEATURE_NAMES = ['round'] + [f'SoR profit {bundles[i]}' for i in range(len(bundles))] \
+ [f'Clock profit {bundles[i]}' for i in range(len(bundles))] + ['activity', 'SoR exposure'] \
+ [f'Price {num_to_letter(i)}' for i in range(num_products)] + [f'Holdings {num_to_letter(i)}' for i in range(num_products)] + [f'Agg {num_to_letter(i)}' for i in range(num_products)]

dot_data = tree.export_graphviz(clf, out_file=None, feature_names=FEATURE_NAMES,  class_names=list(map(str, pd.Series(clusters).unique())), filled=True, rounded=True, special_characters=True)  
graph = graphviz.Source(dot_data) 
graph 



In [ ]:
df.groupby('round')['embedding'].apply(lambda grp: np.array(grp).sum().sum())